<a href="https://colab.research.google.com/github/frankge777/cs521-hw4/blob/main/hw5q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @title Final Optimized Solution (High Precision)
import subprocess
import sys
import re
import torch

# 1. Install Dependencies
def install_packages():
    packages = ["syncode", "transformers", "torch", "accelerate"]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U"] + packages)

try:
    from syncode import Syncode
except ImportError:
    print("Installing dependencies...")
    install_packages()
    from syncode import Syncode
print("Loading SynCode with microsoft/phi-2...")
device = "cuda" if torch.cuda.is_available() else "cpu"

syn_llm = Syncode(
    model="microsoft/phi-2",
    mode="grammar_mask",
    grammar=grammar,
    max_new_tokens=50,
    device=device
)
print("Model loaded.")
def run_syncode(textual_query: str):
    prompt = few_shot_prompt.format(query=textual_query)

    output_list = syn_llm.infer(prompt)
    raw_output = output_list[0]

    # Regex to capture the numbers
    match = re.search(r"Extraction:\s*([\+\-\*\/])\s*(-?\d+(?:\.\d+)?)\s*(-?\d+(?:\.\d+)?)", raw_output)

    if not match:
        return ([], "", 0.0)

    operator = match.group(1)
    op1 = float(match.group(2))
    op2 = float(match.group(3))

    # Calculate
    result = 0.0
    if operator == "+": result = op1 + op2
    elif operator == "-": result = op1 - op2
    elif operator == "*": result = op1 * op2
    elif operator == "/": result = op1 / op2 if op2 != 0 else float('inf')

    return ([op1, op2], operator, result)

# ==========================================
# 5. Test Verification
# ==========================================
if __name__ == "__main__":
    # Test the case that failed before
    q = "What is 50.5 divided by 5?"
    print(f"Query: {q}")
    res = run_syncode(q)
    print(f"Result: {res}")


Loading SynCode with microsoft/phi-2...
[2025-12-11 09:24:25,412-root] - Loading model microsoft/phi-2 with device:cpu, device_map:auto, torch_dtype:torch.bfloat16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[2025-12-11 09:24:46,782-syncode.mask_store.mask_store] - Using cache: True and fsm path cache/mask_stores/CodeGenTokenizerFast/grammar_mask_6844608005_50257.pkl exist: False
[2025-12-11 09:24:46,783-syncode.mask_store.mask_store] - Creating mask store for CodeGenTokenizerFast and custom, may take more than 10 minutes. Caching at /content/cache/mask_stores/CodeGenTokenizerFast/grammar_mask_6844608005_50257.pkl.
[2025-12-11 09:24:47,244-syncode.mask_store.fsm_set] - 8 FSMs with 34 states initialized in 0.00 seconds
[2025-12-11 09:24:47,245-syncode.mask_store.mask_store] - Ignore whitespace is True
[2025-12-11 09:24:47,246-syncode.mask_store.mask_store] - Number of 2 length terminal sequences reduced from 64 to 24


100%|██████████| 34/34 [00:06<00:00,  4.90it/s]

[2025-12-11 09:24:54,233-syncode.mask_store.mask_store] - Time taken to create mask store: 7.44 seconds



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model loaded.
Query: What is 50.5 divided by 5?
Result: ([50.5, 5.0], '/', 10.1)
✅ Success! Precision issue fixed.
